In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import os

In [2]:
data_info = pd.read_excel('data_with_mmse1.xlsx', sheet_name='60以上-1706人')
#data_info = pd.read_excel('./data/data_encoded_del_0724.xlsx')

In [3]:
# data_info = data_info.loc[data_info['del'] != 1]
data_info['name'] = data_info['name'].str.replace('[:,.]', '', regex=True)

In [1]:
import pandas as pd
import os
import shutil
from tqdm import tqdm

# Read the Excel file
data_info = pd.read_excel('./data/data_with_mmse1.xlsx')

# Define the levels list
level_list = [i for i in range(2, 12)]

# Loop over each row in the DataFrame
for id, name in tqdm(data_info[['id', 'name']].values):
    for level in level_list:
        src_file = f'./data/q_splitted_interpolated/Q{level}/{id}-{name}-{level}-S2.csv'
        dest_dir = f'./data/q_splitted_interpolated_del/Q{level}'
        dest_file = os.path.join(dest_dir, f'{id}-{name}-{level}-S2.csv')
        
        # Create the destination directory if it doesn't exist
        os.makedirs(dest_dir, exist_ok=True)
        
        # Copy the file
        if os.path.exists(src_file):
            shutil.copy(src_file, dest_file)
        else:
            print(f"File not found: {src_file}")

print("File copying completed.")

100%|██████████| 1456/1456 [00:02<00:00, 654.99it/s]

File copying completed.


In [4]:
# True is patient, False is normal
def get_diag(row):
    if row["文化程度-更新"] == "中学及以上":
        return row["MMSE得分"] <= 24
    elif row["文化程度-更新"] == "小学":
        return row["MMSE得分"] <= 20
    else:
        return row["MMSE得分"] <= 17

# data_info["MMSE_diag"] = data_info.apply(get_diag, axis=1)

In [5]:
level_list = [i for i in range(2, 12)]
for level in level_list:
    os.makedirs(f'./data/q_splitted_del/Q{level}/', exist_ok=True)

## Question Split

In [6]:
level_list = [i for i in range(2, 12)]
for id, name in tqdm(data_info[['id', 'name']].values):
    eye_data = pd.read_csv(f'./data/eye_data/{id}-{name}.txt')
    for level in level_list:
        level_df = eye_data.loc[(eye_data['level'] == level) & (eye_data['state'] == 2), ['timestamp', 'pos_x', 'pos_y']]
        level_df['timestamp'] = level_df['timestamp'] - level_df['timestamp'].min()
        level_df.fillna(method='ffill', inplace=True)
        level_df.to_csv(f'./data/q_splitted_del/Q{level}/{id}-{name}-{level}-S2.csv')
# for id, name in tqdm(ci_info.values):
#     eye_data = pd.read_csv(f'./data/eye_data/{id}-{name}.txt')
#     for level in level_list:
#         level_df = eye_data.loc[(eye_data['level'] == level) & (eye_data['state'] == 2), ['timestamp', 'pos_x', 'pos_y']]
#         level_df['timestamp'] = level_df['timestamp'] - level_df['timestamp'].min()
#         level_df.to_csv(f'./data/splitted_raw/CI-{id}-{name}-{level}-S2.csv')

100%|██████████| 1456/1456 [01:09<00:00, 20.96it/s]


In [ ]:
# 异常检测，遍历所有的eye data，如空白大于60%则去除所有的下属数据文件
# find /newdata/users/qzy/ET-MMSE/data -type f -name "id-name*" -exec rm {} +

## Question-wise interpolation

In [8]:
level_list = [i for i in range(2, 12)]
for level in level_list:
    time_span = 11 if level == 2 else 7
    try:
        for file in tqdm(os.listdir(f'./data/q_splitted_del/Q{level}/')):
            data = pd.read_csv(f'./data/q_splitted/Q{level}/' + file)
            x = data['pos_x'].values
            y = data['pos_y'].values
            t = data['timestamp'].values
            t_new = np.linspace(0, time_span * 1000, time_span * 70)
            x_new = np.interp(t_new, t, x)
            y_new = np.interp(t_new, t, y)
            interpolated_df = pd.DataFrame([t_new, x_new, y_new]).T
            interpolated_df.columns = ['timestamp', 'x', 'y']
            interpolated_df.to_csv(f'./data/q_splitted_interpolated_del/Q{level}/{file}')
    except Exception as e:
        print(file)
        pass

  0%|          | 0/1456 [00:00<?, ?it/s]


14793-胡长松-2-S2.csv


  0%|          | 0/1456 [00:00<?, ?it/s]


14563-鲍新明-3-S2.csv


  0%|          | 0/1456 [00:00<?, ?it/s]


...

12370-张朝英-9-S2.csv


  0%|          | 0/1456 [00:00<?, ?it/s]


14723-张同惠-10-S2.csv


  0%|          | 0/1456 [00:00<?, ?it/s]

15003-严玉英-11-S2.csv


In [ ]:

def data(self, sub_info: pd.DataFrame, sub_labels: pd.DataFrame, img_size = (64, 64), img_dir: str = './data/scanpath_images') -> None:
    for id, name in sub_info:
        for level in range(2, 12):
            self.transform_data(
                f'{self.img_dir}/Q{level}/{id}_{name}_Q{level}.png',
                pd.read_csv(f'./data/q_splitted_interpolated/Q{level}/{id}-{name}-{level}-S2.csv', usecols=['x', 'y']).values
            )
            
        

def transform_data(self, img_path, positions):
    # Load the image
    save_scanpath_image(positions, img_path) # put it to the first line of transform_data

def save_scanpath_image(positions, save_path, fig_size=(8, 6), dpi=100):  # Default figure size
    os.makedirs(os.path.dirname(save_path), exist_ok=True)  # Create directory if it doesn't exist
    plt.figure(figsize=fig_size, dpi=dpi)  # Set the figure size
    plt.plot(positions[:, 0], positions[:, 1], 'o-', markersize=1, linewidth=0.5)  # Draw the scanpath
    plt.axis('off')  # Turn off the axis
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0)  # Save the figure without extra space
    plt.close()


In [ ]:
data()